# 浸水想定区域データの加工
## 都道府県ごとの浸水想定区域データから任意の水系のshpファイルを作成
- 国土数値情報：浸水想定区域
- http://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-A31.html


In [ ]:
import glob
import re
import os
import pandas as pd

## １）★河川名と都道府県(English)を変数に代入

In [ ]:
# 都道府県
region = "Kanagawa"

In [ ]:
# 抽出する河川
river = "多摩川"

In [ ]:
# 抽出する河川
river_2 = "Tamagawa"

In [ ]:
dir = "../Data/Raw"

## ２）浸水想定区域データの属性ファイル(txt)をリスト(allFiles)に保存
- （参考）https://note.nkmk.me/python-glob-usage/

In [ ]:
allFiles = sorted(glob.glob(dir + "/" + region + "*/*/*.txt"))

In [ ]:
len(allFiles)

In [ ]:
allFiles

## ３）抽出する河川名の入ったファイルをリスト化(list_1)
- （参考）https://note.nkmk.me/python-file-io-open-with/
- （参考）https://note.nkmk.me/python-str-search/
- （参考）https://note.nkmk.me/python-if-elif-else/

In [ ]:
list_1 = []

In [ ]:
for file_ in allFiles:
    with open(file_) as f:
        s = f.read()
        if river in s:
            # ファイル名（５桁の数字）を抽出
            code = f.name[-9:-4]
            list_1.append(code)

In [ ]:
list_1

In [ ]:
# 確認用
with open(dir + "/" + region + "_Tamagawa/属性テキストファイル/" + list_1[0] + ".txt") as f:
        s = f.read()

In [ ]:
s

## ４）浸水想定区域のシェープファイル(shp)を読み込み(data)
- geopandasライブラリを利用
- （参考）https://www.gis-py.com/entry/geopandas-master

In [ ]:
# geopandasをインポート
import geopandas as gpd
from matplotlib import pyplot as plt

In [ ]:
# Jupyter Notebookでノートブック上にグラフを描画する際に指定する記述
%matplotlib inline

## ５）★都道府県コードを変数（code）に代入

In [ ]:
# 都道府県コードを代入
code = "14"

In [ ]:
fp = dir + "/" + region + "_Tamagawa/A31-12_" + code + ".shp"

In [ ]:
fp

In [ ]:
data = gpd.read_file(fp, encoding="SHIFT-JIS")

In [ ]:
data.head()

## ６）"多摩川"の入ったデータ（list_1)を抽出
- （参考）https://note.nkmk.me/python-pandas-concat/

In [ ]:
list_2 = []

In [ ]:
for i in list_1:
    data_1 = data[data.A31_006 == i]
    list_2.append(data_1)
data_2 = pd.concat(list_2)

In [ ]:
data_2.head()

## ７）７段階浸水深ランクを５段階に変換
- （参考）https://qiita.com/Hyperion13fleet/items/98c31744e66ac1fc1e9f

In [ ]:
# 条件分岐の関数を定義
def func_cat(x):
    if x ==21:
        return 11
    elif x == 22:
        return 12
    elif x == 23:
        return 13
    elif x == 24:
        return 14
    elif x == 25:
        return 14
    elif x == 26:
        return 14
    elif x == 27:
        return 15
    else:
        return x

# applyで適用
data_2["Shinsui"] = data_2["A31_001"].apply(func_cat)  

In [ ]:
data_2

In [ ]:
data_2.dtypes

In [ ]:
type(data_2)

## ８）Fionaを使用して測地系の設定
- （参考）https://www.gis-py.com/entry/geopandas-master

In [ ]:
from fiona.crs import from_epsg

In [ ]:
data_2.crs = from_epsg(2451)

In [ ]:
data_2.crs

## ９）shpファイルに書き出し

- 抽出する河川名のファイルの要素数あるリストを作成
- （参考）http://nekoyukimmm.hatenablog.com/entry/2016/02/26/091217

In [ ]:
dir_1 = "../Data/Working"

In [ ]:
dir_2 = dir_1 + "/" + region + "_" + river_2

In [ ]:
dir_2

In [ ]:
# 作成したデータ格納用にディレクトリを作成
os.mkdir(dir_2)

In [ ]:
data_2.to_file(dir_2, encoding='UTF-8')

In [ ]:
data_2.plot()